## Aspect Term Extraction (ATE) Training and Fine Tuning for Large Language Models on German hospital reviews using OBI-Tagging

In [1]:
import torch
import os

import spacy
import ast  # To safely evaluate strings as Python objects

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import evaluate

from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

# We need the sys package to load modules from another directory:
import sys
sys.path.append('../')
from functions.ate_model_train import *

/home/sc.uni-leipzig.de/ch31qoni/venv/absa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Is CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

Is CUDA available: True
CUDA version: 12.6
GPU device name: NVIDIA A30


In [3]:
# Load the dataset into a DataFrame
data = pd.read_csv("./data/hospitalABSA/patient_review_labels_absa.csv")
data_ano = pd.read_csv("./data/hospitalABSA/patient_review_labels_absa_ano.csv")

In [4]:
models = ["google-bert/bert-base-german-cased","dbmdz/bert-base-german-cased", "dbmdz/bert-base-german-uncased",
          "FacebookAI/xlm-roberta-base", "TUM/GottBERT_base_best", "TUM/GottBERT_filtered_base_best", "TUM/GottBERT_base_last",
          "distilbert/distilbert-base-german-cased", "GerMedBERT/medbert-512", "deepset/gbert-base"]


### 1. standard ATE

In [5]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=10)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4797.18 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_v2.py:416: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['I-ASPECT' 'O' 'B-ASPECT']
{0: 21.94744744744745, 1: 0.3725025484199796, 2: 3.705196451204056}
Training results for google-bert/bert-base-german-cased with 10 epochs and random seeds: 42, 42



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.252091,0.753363,0.736842,0.745011
2,0.611200,0.206167,0.777778,0.828947,0.802548
3,0.175900,0.289843,0.792035,0.785088,0.788546
4,0.074300,0.253312,0.788136,0.815789,0.801724
5,0.050900,0.346724,0.807018,0.807018,0.807018
6,0.050900,0.440677,0.821101,0.785088,0.802691
7,0.021500,0.445545,0.795745,0.820175,0.807775
8,0.009200,0.500903,0.804348,0.811404,0.807860
9,0.004100,0.520252,0.807018,0.807018,0.807018
10,0.002300,0.526721,0.804348,0.811404,0.807860


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4359.72 examples/s]


Unique predicted label IDs: {0, 1, 2}
Expected label IDs: {0, 1, 2}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.85      0.74      0.80       261

   micro avg       0.85      0.74      0.80       261
   macro avg       0.85      0.74      0.80       261
weighted avg       0.85      0.74      0.80       261

Precision Score: 0.8546255506607929
Recall Score: 0.7432950191570882
F1 Score: 0.7950819672131147
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pre

Model training with 5 epochs and f1 to load the best model

In [5]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=5)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 3753.38 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_v2.py:416: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['O' 'B-ASPECT' 'I-ASPECT']
{0: 0.3725025484199796, 1: 3.705196451204056, 2: 21.94744744744745}
Training results for google-bert/bert-base-german-cased with 5 epochs and random seeds: 42, 42



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.257658,0.719828,0.732456,0.726087
2,0.619300,0.225630,0.764228,0.824561,0.793249
3,0.174400,0.246719,0.792952,0.789474,0.791209
4,0.075100,0.251072,0.788793,0.802632,0.795652
5,0.055900,0.319447,0.805195,0.815789,0.810458


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 3471.77 examples/s]


Unique predicted label IDs: {0, 1, 2}
Expected label IDs: {0, 1, 2}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.83      0.72      0.77       261

   micro avg       0.83      0.72      0.77       261
   macro avg       0.83      0.72      0.77       261
weighted avg       0.83      0.72      0.77       261

Precision Score: 0.8289473684210527
Recall Score: 0.7241379310344828
F1 Score: 0.7730061349693252
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pre

In [6]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=7)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4642.36 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_v2.py:416: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['I-ASPECT' 'O' 'B-ASPECT']
{0: 21.94744744744745, 1: 0.3725025484199796, 2: 3.705196451204056}
Training results for google-bert/bert-base-german-cased with 7 epochs and random seeds: 42, 42



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.252348,0.721311,0.771930,0.745763
2,0.622400,0.235447,0.771300,0.754386,0.762749
3,0.173400,0.284240,0.783186,0.776316,0.779736
4,0.072400,0.307109,0.785088,0.785088,0.785088
5,0.057100,0.390860,0.791111,0.780702,0.785872
6,0.057100,0.407471,0.787879,0.798246,0.793028
7,0.023100,0.448715,0.796460,0.789474,0.792952


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4458.91 examples/s]


Unique predicted label IDs: {0, 1, 2}
Expected label IDs: {0, 1, 2}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.84      0.73      0.78       261

   micro avg       0.84      0.73      0.78       261
   macro avg       0.84      0.73      0.78       261
weighted avg       0.84      0.73      0.78       261

Precision Score: 0.8377192982456141
Recall Score: 0.7318007662835249
F1 Score: 0.7811860940695297
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pre

In [7]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=8)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4592.44 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_v2.py:416: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['I-ASPECT' 'O' 'B-ASPECT']
{0: 21.94744744744745, 1: 0.3725025484199796, 2: 3.705196451204056}
Training results for google-bert/bert-base-german-cased with 8 epochs and random seeds: 42, 42



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.240540,0.738739,0.719298,0.728889
2,0.614500,0.228364,0.748000,0.820175,0.782427
3,0.170300,0.240929,0.796537,0.807018,0.801743
4,0.071400,0.246625,0.798283,0.815789,0.806941
5,0.043100,0.346854,0.786008,0.837719,0.811040
6,0.043100,0.375973,0.804348,0.811404,0.807860
7,0.018400,0.431386,0.817778,0.807018,0.812362
8,0.009300,0.437655,0.802575,0.820175,0.811280


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4398.26 examples/s]


Unique predicted label IDs: {0, 1, 2}
Expected label IDs: {0, 1, 2}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.85      0.72      0.78       261

   micro avg       0.85      0.72      0.78       261
   macro avg       0.85      0.72      0.78       261
weighted avg       0.85      0.72      0.78       261

Precision Score: 0.8506787330316742
Recall Score: 0.7203065134099617
F1 Score: 0.7800829875518672
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pre

In [6]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=6)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 3509.38 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_v2.py:416: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['O' 'B-ASPECT' 'I-ASPECT']
{0: 0.3725025484199796, 1: 3.705196451204056, 2: 21.94744744744745}
Training results for google-bert/bert-base-german-cased with 6 epochs and random seeds: 42, 42



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.243179,0.735426,0.719298,0.727273
2,0.621300,0.223387,0.762115,0.758772,0.760440
3,0.171800,0.256678,0.795455,0.767544,0.781250
4,0.074500,0.243001,0.796537,0.807018,0.801743
5,0.044000,0.363324,0.794643,0.780702,0.787611
6,0.044000,0.388545,0.809091,0.780702,0.794643


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 3409.03 examples/s]


Unique predicted label IDs: {0, 1, 2}
Expected label IDs: {0, 1, 2}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.80      0.74      0.77       261

   micro avg       0.80      0.74      0.77       261
   macro avg       0.80      0.74      0.77       261
weighted avg       0.80      0.74      0.77       261

Precision Score: 0.8
Recall Score: 0.735632183908046
F1 Score: 0.7664670658682634
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 

In [8]:
for model in models:
    print(f'training and results for {model}:')
    ate_model(data, model, rn1=42, rn2=42, epochs=12)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mapping of the data



Map: 100%|██████████| 101/101 [00:00<00:00, 4603.77 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/ate_model_train_v2.py:416: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['I-ASPECT' 'O' 'B-ASPECT']
{0: 21.94744744744745, 1: 0.3725025484199796, 2: 3.705196451204056}
Training results for google-bert/bert-base-german-cased with 12 epochs and random seeds: 42, 42



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.243553,0.742489,0.758772,0.750542
2,0.612200,0.226384,0.753138,0.789474,0.770878
3,0.178900,0.259738,0.780172,0.793860,0.786957
4,0.071500,0.279910,0.797414,0.811404,0.804348
5,0.049100,0.356831,0.784232,0.828947,0.805970
6,0.049100,0.445217,0.807339,0.771930,0.789238
7,0.017000,0.477910,0.781893,0.833333,0.806794
8,0.008900,0.492423,0.809955,0.785088,0.797327
9,0.007100,0.541927,0.806867,0.824561,0.815618
10,0.005000,0.550221,0.773279,0.837719,0.804211


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


mapping the test data



Map: 100%|██████████| 102/102 [00:00<00:00, 4383.53 examples/s]


Unique predicted label IDs: {0, 1, 2}
Expected label IDs: {0, 1, 2}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.86      0.75      0.80       261

   micro avg       0.86      0.75      0.80       261
   macro avg       0.86      0.75      0.80       261
weighted avg       0.86      0.75      0.80       261

Precision Score: 0.8558951965065502
Recall Score: 0.7509578544061303
F1 Score: 0.8
Tokens     : ['Nun', 'sind', '3', 'Jahre', 'seit', 'der', 'Operation', 'vergangen', 'und', 'es', 'mir', 'gut', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O']
Tokens     : ['Insbesondere', 'bei', 'Unikliniken', ',', 'mit', 'anderen', 'Krankheitsbildern', 'haben', 'sie', 'leider', 'ab', 'und', 'zu', 'Probleme', '.']
True Labels: ['O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O',

### 2. category-aware ATE

In [11]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=5)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 72/72 [00:00<00:00, 3904.81 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/model_train_v2.py:669: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['I-anderer Service' 'B-anderer Service' 'B-mediz. Service' 'I-Personal'
 'O' 'I-Pflegepersonal' 'B-Krankenhaus' 'B-Arzt' 'I-mediz. Service'
 'I-Arzt' 'B-Pflegepersonal' 'B-Personal' 'I-Krankenhaus']
{0: np.float64(78.56153846153846), 1: np.float64(5.651909241837299), 2: np.float64(2.846432552954292), 3: np.float64(71.41958041958041), 4: np.float64(0.08550450420280634), 5: np.float64(261.87179487179486), 6: np.float64(3.9280769230769232), 7: np.float64(5.493813878429263), 8: np.float64(23.806526806526808), 9: np.float64(18.705128205128204), 10: np.float64(9.465245597775718), 11: np.float64(13.545092838196286), 12: np.float64(29.096866096866098)}
Training google-bert/bert-base-german-cased for 5 epochs with random seeds 42, 42



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.564667,0.338983,0.372671,0.355030
2,2.451800,0.410399,0.564885,0.459627,0.506849
3,2.451800,0.458173,0.502703,0.577640,0.537572
4,0.205300,0.514803,0.662162,0.608696,0.634304
5,0.205300,0.548273,0.616352,0.608696,0.612500


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


Evaluating on test data



Map: 100%|██████████| 72/72 [00:00<00:00, 4009.85 examples/s]


Unique predicted label IDs: {np.int64(0), np.int64(3), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(11), np.int64(13)}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.52      0.58      0.55        24
    Krankenhaus       0.86      0.80      0.83        45
       Personal       0.67      0.75      0.71         8
 Pflegepersonal       0.91      0.91      0.91        11
anderer Service       0.33      0.38      0.35        16
 mediz. Service       0.56      0.32      0.41        31

      micro avg       0.66      0.61      0.63       135
      macro avg       0.64      0.62      0.63       135
   weighted avg       0.66      0.61      0.62       135

Precision Score: 0.656
Recall Score: 0.6074074074074074
F1 Score: 0.6307692307692309
Tokens     : ['Ich', 'habe', 'jeden', 'Tag', 'die', 'Stationsärztin', 'gesehen', ',', 'hier', 'nochm

In [12]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model(data, model, rn1=42, rn2=42, epochs=10)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Mapping the data


Map: 100%|██████████| 72/72 [00:00<00:00, 3823.09 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/model_train_v2.py:669: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['I-anderer Service' 'B-anderer Service' 'B-mediz. Service' 'I-Personal'
 'O' 'I-Pflegepersonal' 'B-Krankenhaus' 'B-Arzt' 'I-mediz. Service'
 'I-Arzt' 'B-Pflegepersonal' 'B-Personal' 'I-Krankenhaus']
{0: np.float64(78.56153846153846), 1: np.float64(5.651909241837299), 2: np.float64(2.846432552954292), 3: np.float64(71.41958041958041), 4: np.float64(0.08550450420280634), 5: np.float64(261.87179487179486), 6: np.float64(3.9280769230769232), 7: np.float64(5.493813878429263), 8: np.float64(23.806526806526808), 9: np.float64(18.705128205128204), 10: np.float64(9.465245597775718), 11: np.float64(13.545092838196286), 12: np.float64(29.096866096866098)}
Training google-bert/bert-base-german-cased for 10 epochs with random seeds 42, 42



...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.560103,0.356757,0.409938,0.381503
2,2.415900,0.378509,0.578571,0.503106,0.538206
3,2.415900,0.459142,0.518919,0.596273,0.554913
4,0.206600,0.542074,0.600000,0.596273,0.598131
5,0.206600,0.651896,0.593548,0.571429,0.582278
6,0.073700,0.802876,0.576923,0.559006,0.567823
7,0.024100,0.756369,0.617450,0.571429,0.593548
8,0.024100,0.822646,0.573171,0.583851,0.578462
9,0.008800,0.823921,0.567251,0.602484,0.584337
10,0.008800,0.837292,0.592593,0.596273,0.594427


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


Evaluating on test data



Map: 100%|██████████| 72/72 [00:00<00:00, 4051.27 examples/s]


Unique predicted label IDs: {np.int64(0), np.int64(3), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(11), np.int64(13)}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.52      0.62      0.57        24
    Krankenhaus       0.73      0.73      0.73        45
       Personal       0.55      0.75      0.63         8
 Pflegepersonal       0.67      0.91      0.77        11
anderer Service       0.30      0.38      0.33        16
 mediz. Service       0.65      0.42      0.51        31

      micro avg       0.59      0.61      0.60       135
      macro avg       0.57      0.64      0.59       135
   weighted avg       0.61      0.61      0.60       135

Precision Score: 0.5928571428571429
Recall Score: 0.6148148148148148
F1 Score: 0.6036363636363636
Tokens     : ['Ich', 'habe', 'jeden', 'Tag', 'die', 'Stationsärztin', 'gesehen', ',', '

#### 2.1 category-aware ATE with k-fold cross validation

In [10]:
for model in models:
    print(f'training and results for {model}:')
    ate_cat_model_kfold(data, model, rn1=42, rn2=42, k=2, epochs=5)
    print()

training and results for google-bert/bert-base-german-cased:


Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting fold 1/2


Map: 100%|██████████| 180/180 [00:00<00:00, 5902.21 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/model_train_v2.py:791: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['I-Pflegepersonal' 'B-Arzt' 'B-anderer Service' 'I-Personal' 'O'
 'B-mediz. Service' 'I-anderer Service' 'I-Arzt' 'I-mediz. Service'
 'B-Krankenhaus' 'I-Krankenhaus' 'B-Pflegepersonal' 'B-Personal']
{0: np.float64(261.87179487179486), 1: np.float64(5.493813878429263), 2: np.float64(5.651909241837299), 3: np.float64(71.41958041958041), 4: np.float64(0.08550450420280634), 5: np.float64(2.846432552954292), 6: np.float64(78.56153846153846), 7: np.float64(18.705128205128204), 8: np.float64(23.806526806526808), 9: np.float64(3.9280769230769232), 10: np.float64(29.096866096866098), 11: np.float64(9.465245597775718), 12: np.float64(13.545092838196286)}
Training fold 1


...

There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


Evaluating fold 2


Unique predicted label IDs: {np.int64(0), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(11)}
Expected label IDs: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
Classification Report:
                 precision    recall  f1-score   support

           Arzt       0.49      0.69      0.57        42
    Krankenhaus       0.41      0.74      0.53        96
       Personal       0.00      0.00      0.00        22
 Pflegepersonal       0.54      0.77      0.63        48
anderer Service       0.00      0.00      0.00        55
 mediz. Service       0.45      0.65      0.53       106

      micro avg       0.46      0.56      0.50       369
      macro avg       0.32      0.48      0.38       369
   weighted avg       0.36      0.56      0.44       369

Fold 2 Results - Precision: 0.4557522123893805, Recall: 0.5582655826558266, F1: 0.5018270401948843

=== Final Cross-Validation Results ===
Average Precision: 0.260839326180811
Average Recall: 0.4064786358319348
A

/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training complete. Model directory for fold 1 deleted to free memory.
Training complete. Model directory for fold 2 deleted to free memory.



In [6]:
ate_model(data, "GerMedBERT/medbert-512", rn1=42, rn2=42, epochs=7)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Device set to use cuda:0


mapping of the data



Map: 100%|██████████| 72/72 [00:00<00:00, 4142.07 examples/s]
/home/sc.uni-leipzig.de/ch31qoni/venv/py312_2/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/sc.uni-leipzig.de/ch31qoni/ABSA/model_train_v2.py:414: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['O' 'B-ASPECT' 'I-ASPECT']
{0: np.float64(0.3705195182121608), 1: np.float64(3.78680014831294), 2: np.float64(27.01851851851852)}
Training results for GerMedBERT/medbert-512 with 7 epochs and random seeds: 42, 42



Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.302630,0.657534,0.596273,0.625407
2,2.032800,0.241518,0.691781,0.627329,0.657980
3,2.032800,0.276279,0.671875,0.534161,0.595156
4,0.148300,0.434866,0.620112,0.689441,0.652941
5,0.148300,0.502979,0.658385,0.658385,0.658385
6,0.046400,0.574677,0.654545,0.670807,0.662577
7,0.009500,0.630149,0.634286,0.689441,0.660714


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


mapping the test data



Map: 100%|██████████| 72/72 [00:00<00:00, 4259.98 examples/s]


Unique predicted label IDs: {np.int64(0), np.int64(1), np.int64(2)}
Expected label IDs: {0, 1, 2}
Classification Report:
              precision    recall  f1-score   support

      ASPECT       0.69      0.65      0.67       135

   micro avg       0.69      0.65      0.67       135
   macro avg       0.69      0.65      0.67       135
weighted avg       0.69      0.65      0.67       135

Precision Score: 0.6875
Recall Score: 0.6518518518518519
F1 Score: 0.6692015209125476
Tokens     : ['Ich', 'habe', 'jeden', 'Tag', 'die', 'Stationsärztin', 'gesehen', ',', 'hier', 'nochmal', 'vielen', 'Dank', 'für', 'die', 'Geduld', 'und', 'die', 'fachliche', 'Betreuung', 'und', 'die', 'Empathie', 'die', 'mir', 'entgegengebracht', 'wurde', '.']
True Labels: ['O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O']
Pred Labels: ['O', 'O', 'O', 'O', 'O', 'B-ASPECT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'